# Phase 8: Advanced Instruments & Hedging

Derivatives and complex instruments for sophisticated leverage management.

This notebook covers:
- **Options basics** — calls, puts, Greeks
- **Hedging with derivatives** — protective puts, covered calls
- **Ratio-backspreads and ZEBRAs** — asymmetric risk/reward
- **Leveraged ETFs** — understanding decay and rebalancing

---

```bash
pip install pandas numpy matplotlib scipy
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

---
## 8.1 Options Pricing (Black-Scholes)

Understanding options is essential for advanced hedging and leverage strategies.

In [ ]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    """Black-Scholes option pricing.
    S: spot price
    K: strike price
    T: time to expiration (years)
    r: risk-free rate
    sigma: volatility
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100  # per 1% vol change
    theta = -(S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) / 365  # per day
    
    return {'price': price, 'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta}

# Example: ATM call option
S = 100  # spot
K = 100  # strike (at-the-money)
T = 30/365  # 30 days to expiration
r = 0.05  # 5% risk-free rate
sigma = 0.25  # 25% implied volatility

call = black_scholes(S, K, T, r, sigma, 'call')
put = black_scholes(S, K, T, r, sigma, 'put')

print(f"=== ATM Options (S={S}, K={K}, T={T*365:.0f}d, σ={sigma:.0%}) ===")
print(f"\nCall option:")
for k, v in call.items():
    print(f"  {k}: {v:.4f}")
print(f"\nPut option:")
for k, v in put.items():
    print(f"  {k}: {v:.4f}")

print(f"\nPut-Call Parity check: C - P = {call['price'] - put['price']:.4f}, S - K*e^(-rT) = {S - K*np.exp(-r*T):.4f}")

In [ ]:
# Visualize option payoffs and P&L
spot_range = np.linspace(80, 120, 100)

call_premium = call['price']
put_premium = put['price']

# Payoffs at expiration
call_payoff = np.maximum(spot_range - K, 0) - call_premium
put_payoff = np.maximum(K - spot_range, 0) - put_premium
stock_payoff = spot_range - S

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# P&L diagram
axes[0].plot(spot_range, call_payoff, color='green', linewidth=2, label=f'Long Call (cost ${call_premium:.2f})')
axes[0].plot(spot_range, put_payoff, color='red', linewidth=2, label=f'Long Put (cost ${put_premium:.2f})')
axes[0].plot(spot_range, stock_payoff, color='blue', linestyle='--', label='Long Stock')
axes[0].axhline(y=0, color='gray', linewidth=0.5)
axes[0].axvline(x=K, color='gray', linestyle=':', alpha=0.5)
axes[0].set_xlabel('Spot Price at Expiration ($)')
axes[0].set_ylabel('Profit/Loss ($)')
axes[0].set_title('Option P&L at Expiration', fontsize=13)
axes[0].legend()
axes[0].set_ylim(-15, 25)

# Delta across spot prices
deltas_call = [black_scholes(s, K, T, r, sigma, 'call')['delta'] for s in spot_range]
deltas_put = [black_scholes(s, K, T, r, sigma, 'put')['delta'] for s in spot_range]

axes[1].plot(spot_range, deltas_call, color='green', linewidth=2, label='Call Delta')
axes[1].plot(spot_range, deltas_put, color='red', linewidth=2, label='Put Delta')
axes[1].axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
axes[1].axhline(y=-0.5, color='gray', linestyle=':', alpha=0.5)
axes[1].axvline(x=K, color='gray', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Spot Price ($)')
axes[1].set_ylabel('Delta')
axes[1].set_title('Option Delta vs Spot Price', fontsize=13)
axes[1].legend()

plt.tight_layout()
plt.show()

---
## 8.2 Hedging with Options

Use options to protect leveraged positions from downside risk.

In [ ]:
def simulate_hedged_portfolio(n_days=60, leverage=3.0, hedge_type='none'):
    """Simulate leveraged portfolio with optional option hedge."""
    np.random.seed(42)
    
    # Generate price path
    S0 = 100
    mu = 0.10
    sigma = 0.25
    dt = 1/252
    
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n_days)
    prices = S0 * np.exp(np.cumsum(returns))
    prices = np.insert(prices, 0, S0)
    
    # Portfolio value
    equity = 100_000
    position_value = equity * leverage
    shares = position_value / S0
    
    # Option hedge costs (if applicable)
    hedge_cost = 0
    put_strike = S0 * 0.95  # 5% OTM put
    put_info = black_scholes(S0, put_strike, n_days/252, 0.05, sigma, 'put')
    
    if hedge_type == 'protective_put':
        # Buy puts to cover the leveraged position
        hedge_cost = put_info['price'] * shares
    elif hedge_type == 'collar':
        # Buy put, sell call (zero-cost collar)
        call_strike = S0 * 1.05
        call_info = black_scholes(S0, call_strike, n_days/252, 0.05, sigma, 'call')
        hedge_cost = (put_info['price'] - call_info['price']) * shares
    
    # Track portfolio through time
    portfolio_values = [equity - hedge_cost]
    
    for i in range(1, len(prices)):
        price_return = (prices[i] - prices[i-1]) / prices[i-1]
        
        # Leveraged stock P&L
        stock_pnl = shares * (prices[i] - prices[i-1])
        
        # Option payoff at expiration (only on last day)
        if i == len(prices) - 1:
            if hedge_type == 'protective_put':
                put_payoff = max(put_strike - prices[i], 0) * shares
                stock_pnl += put_payoff
            elif hedge_type == 'collar':
                put_payoff = max(put_strike - prices[i], 0) * shares
                call_payoff = -max(prices[i] - call_strike, 0) * shares  # short call
                stock_pnl += put_payoff + call_payoff
        
        new_value = portfolio_values[-1] + stock_pnl
        portfolio_values.append(new_value)
    
    return np.array(portfolio_values), prices, hedge_cost

# Compare hedged vs unhedged
no_hedge, prices, _ = simulate_hedged_portfolio(hedge_type='none')
with_put, _, put_cost = simulate_hedged_portfolio(hedge_type='protective_put')
with_collar, _, collar_cost = simulate_hedged_portfolio(hedge_type='collar')

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

ax1.plot(prices, color='steelblue', linewidth=1.5)
ax1.set_ylabel('Stock Price ($)')
ax1.set_title('Hedged vs Unhedged Leveraged Portfolio (3x)', fontsize=14)

ax2.plot(no_hedge, color='red', linewidth=1.5, label='No Hedge')
ax2.plot(with_put, color='green', linewidth=1.5, label=f'Protective Put (cost ${put_cost:,.0f})')
ax2.plot(with_collar, color='blue', linewidth=1.5, label=f'Collar (cost ${collar_cost:,.0f})')
ax2.axhline(y=100_000, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Portfolio Value ($)')
ax2.set_xlabel('Day')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Final values:")
print(f"  No hedge:       ${no_hedge[-1]:,.2f} ({(no_hedge[-1]/100_000-1):.1%})")
print(f"  Protective put: ${with_put[-1]:,.2f} ({(with_put[-1]/100_000-1):.1%})")
print(f"  Collar:         ${with_collar[-1]:,.2f} ({(with_collar[-1]/100_000-1):.1%})")

---
## 8.3 Ratio Backspreads

A ratio backspread gives unlimited upside with limited downside — ideal for leveraged directional bets.

**Call Ratio Backspread**: Sell 1 lower-strike call, buy 2 higher-strike calls.

In [ ]:
def ratio_backspread_payoff(spot_range, S, K1, K2, T, r, sigma, ratio=2):
    """1:2 Call ratio backspread: sell 1 K1 call, buy 2 K2 calls (K2 > K1)."""
    # Premiums
    short_call = black_scholes(S, K1, T, r, sigma, 'call')
    long_call = black_scholes(S, K2, T, r, sigma, 'call')
    
    net_premium = short_call['price'] - ratio * long_call['price']
    
    # Payoff at expiration
    short_payoff = -np.maximum(spot_range - K1, 0)
    long_payoff = ratio * np.maximum(spot_range - K2, 0)
    total_payoff = short_payoff + long_payoff + net_premium
    
    return total_payoff, net_premium, short_call['price'], long_call['price']

S = 100
K1 = 100  # ATM (sell)
K2 = 105  # OTM (buy 2x)
T = 30/365
r = 0.05
sigma = 0.25

spot_range = np.linspace(85, 130, 200)
backspread_payoff, net_prem, short_prem, long_prem = ratio_backspread_payoff(
    spot_range, S, K1, K2, T, r, sigma, ratio=2
)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(spot_range, backspread_payoff, color='green', linewidth=2, label='1:2 Call Backspread')
ax.fill_between(spot_range, backspread_payoff, 0, where=backspread_payoff > 0, color='green', alpha=0.2)
ax.fill_between(spot_range, backspread_payoff, 0, where=backspread_payoff < 0, color='red', alpha=0.2)
ax.axhline(y=0, color='gray', linewidth=0.5)
ax.axvline(x=K1, color='gray', linestyle=':', alpha=0.5, label=f'K1={K1} (short)')
ax.axvline(x=K2, color='blue', linestyle=':', alpha=0.5, label=f'K2={K2} (long 2x)')
ax.axvline(x=S, color='orange', linestyle='--', alpha=0.5, label=f'Spot={S}')

ax.set_xlabel('Spot Price at Expiration ($)')
ax.set_ylabel('Profit/Loss ($)')
ax.set_title(f'Call Ratio Backspread (1:{2}) — Net Premium: ${net_prem:.2f}', fontsize=14)
ax.legend()

plt.tight_layout()
plt.show()

print(f"Short 1x {K1} call @ ${short_prem:.2f}")
print(f"Long 2x {K2} call @ ${long_prem:.2f} each (${2*long_prem:.2f} total)")
print(f"Net premium: ${net_prem:.2f} ({'credit' if net_prem > 0 else 'debit'})")
print(f"\nMax loss at {K2}: ${backspread_payoff[np.argmin(np.abs(spot_range - K2))]:.2f}")
print(f"Breakeven points: below {K1} (if credit) or above ~{K2 + abs(net_prem) + (K2-K1):.1f}")

---
## 8.4 Leveraged ETF Mechanics

Leveraged ETFs (e.g., 3x SPY) rebalance daily. This causes **volatility decay** in choppy markets.

In [ ]:
def simulate_leveraged_etf(returns, leverage=3):
    """Simulate a daily-rebalanced leveraged ETF."""
    etf_returns = returns * leverage
    etf_value = (1 + etf_returns).cumprod()
    return etf_value

def simulate_static_leverage(returns, leverage=3):
    """Simulate static leverage (borrow and hold, no rebalancing)."""
    # With static leverage, you'd get leverage * total return
    # But we need to account for daily compounding differently
    underlying = (1 + returns).cumprod()
    # Static leverage: final = 1 + leverage * (underlying_final - 1)
    # But tracking daily: need to track margin
    static_value = [1.0]
    for r in returns:
        new_val = static_value[-1] * (1 + leverage * r)
        static_value.append(new_val)
    return np.array(static_value[1:])

# Simulate different market conditions
n = 252

# Trending market
np.random.seed(42)
trending_returns = np.random.normal(0.0008, 0.01, n)  # low vol, positive drift

# Choppy market (high vol, no drift)
np.random.seed(123)
choppy_returns = np.random.normal(0.0, 0.025, n)  # high vol, no drift

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, returns, title in [(axes[0], trending_returns, 'Trending Market'),
                            (axes[1], choppy_returns, 'Choppy Market')]:
    underlying = (1 + returns).cumprod()
    etf_3x = simulate_leveraged_etf(returns, 3)
    
    ax.plot(underlying, color='gray', linestyle='--', label='1x (underlying)')
    ax.plot(etf_3x, color='red', linewidth=1.5, label='3x Leveraged ETF')
    ax.plot(underlying ** 3, color='blue', linestyle=':', label='Underlying^3 (ideal 3x)')
    ax.axhline(y=1.0, color='gray', alpha=0.3)
    ax.set_xlabel('Day')
    ax.set_ylabel('Value')
    ax.set_title(f'{title}', fontsize=13)
    ax.legend(fontsize=9)
    
    print(f"{title}:")
    print(f"  Underlying: {underlying[-1]:.2%}")
    print(f"  3x ETF:     {etf_3x[-1]:.2%}")
    print(f"  Ideal 3x:   {underlying[-1]**3:.2%}")
    print(f"  Decay:      {(etf_3x[-1] - underlying[-1]**3):.2%}")
    print()

plt.tight_layout()
plt.show()

In [ ]:
# Volatility decay formula
def leveraged_etf_expected_return(mu, sigma, leverage, T=1):
    """Expected return of a leveraged ETF over time T.
    
    The expected log return is:
    E[log(ETF)] = L*mu - 0.5*L^2*sigma^2
    
    vs underlying:
    E[log(underlying)] = mu - 0.5*sigma^2
    """
    underlying_growth = mu - 0.5 * sigma**2
    etf_growth = leverage * mu - 0.5 * (leverage * sigma)**2
    decay = leverage * underlying_growth - etf_growth
    return {
        'underlying_log_return': underlying_growth * T,
        'etf_log_return': etf_growth * T,
        'volatility_decay': decay * T,
        'decay_formula': f"0.5 * {leverage}^2 * ({leverage}-1) * σ^2 = {0.5 * leverage**2 * (leverage-1) * sigma**2:.4f}"
    }

# Show decay for different volatilities
vols = np.arange(0.10, 0.60, 0.05)
decays_2x = [leveraged_etf_expected_return(0.10, v, 2)['volatility_decay'] for v in vols]
decays_3x = [leveraged_etf_expected_return(0.10, v, 3)['volatility_decay'] for v in vols]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(vols * 100, [d * 100 for d in decays_2x], color='blue', marker='o', label='2x ETF')
ax.plot(vols * 100, [d * 100 for d in decays_3x], color='red', marker='s', label='3x ETF')
ax.set_xlabel('Annualized Volatility (%)')
ax.set_ylabel('Annual Volatility Decay (%)')
ax.set_title('Leveraged ETF Volatility Decay vs Underlying Volatility', fontsize=13)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("At 25% volatility:")
for lev in [2, 3]:
    result = leveraged_etf_expected_return(0.10, 0.25, lev)
    print(f"  {lev}x ETF annual decay: {result['volatility_decay']:.2%}")

### Exercise 8.4

1. Simulate a 3x leveraged ETF over 5 years in a market with 10% annual return and 30% vol. What's the total decay?
2. Is it ever better to use a leveraged ETF vs borrowing to buy more shares? When?
3. Design a "rebalancing" strategy that only uses the 3x ETF when the market is trending (low vol) and switches to 1x when choppy.

In [ ]:
# YOUR CODE HERE


---
## 8.5 Comprehension Check

1. A call option has delta=0.6. If the stock moves up $1, approximately how much does the option price change?
2. Why is a protective put like "insurance"? What's the cost of this insurance?
3. A call ratio backspread profits from big moves up and stays flat for small moves. Why would you choose this over just buying calls?
4. A 3x leveraged ETF is down 10% while the underlying is down 3%. Is this expected? Why?
5. You expect a stock to move significantly but aren't sure which direction. What options strategy would you use?

In [ ]:
# YOUR ANSWERS HERE
